In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os, shutil
import zipfile

In [ ]:
labels_csv = pd.read_csv('train.csv')

In [ ]:
labels_csv

In [ ]:
df = labels_csv['Target']

In [ ]:
df = df.str.split(' ')

In [ ]:
labels = []
for i in df:
    y = np.zeros(28)
    for j in i:
        y[int(j)] = 1
    labels.append(y)

In [ ]:
labels_csv = pd.concat([labels_csv['Id'], pd.DataFrame(labels)], axis=1)

In [ ]:
labels_csv

In [ ]:
shutil.rmtree('train')

In [ ]:
with zipfile.ZipFile('train.zip', 'r') as zip_ref:
    zip_ref.extractall('train')

In [ ]:
os.chdir('train')

In [ ]:
os.getcwd()

In [ ]:
colors = ['red', 'green', 'blue', 'yellow']

In [ ]:
for i in labels_csv['Id']:
    os.mkdir(i)
    for j in colors:
        img = str(i) + '_' + str(j) +'.png'
        path = i
        shutil.move(img, path)

In [ ]:
os.chdir('..')

In [ ]:
x = labels_csv['Id']

In [ ]:
y = pd.DataFrame(labels_csv.iloc[:, 1:])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.125)

In [ ]:
os.chdir('train')

In [ ]:
os.mkdir('train')
os.mkdir('valid')

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_valid

In [ ]:
y_valid

In [ ]:
for i in x_train:
    if os.path.isdir(i):
        try:
            shutil.move(i, 'train')
        except:
            pass

for i in x_valid:
    if os.path.isdir(i):
        try:
            shutil.move(i, 'valid')
        except:
            pass

In [ ]:
os.chdir('..')

In [ ]:
os.chdir('train')

In [ ]:
os.getcwd()

In [ ]:
input_size = 352
img_group = 4

In [ ]:
import keras

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255)

valid_datagen = ImageDataGenerator(
    rescale=1./255)

test_datagen = ImageDataGenerator(
    rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory="train/",
    target_size=(input_size, input_size),
    color_mode="grayscale",
    batch_size=img_group,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

In [ ]:
train_labels = list(train_generator.class_indices)

In [ ]:
valid_generator = valid_datagen.flow_from_directory(
    directory="valid/",
    target_size=(input_size, input_size),
    color_mode="grayscale",
    batch_size=img_group,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

In [ ]:
valid_labels = list(valid_generator.class_indices)

In [ ]:
os.getcwd()

In [ ]:
from keras.layers import Conv2D, LeakyReLU, Concatenate, ReLU, Input, Dense, Dropout, BatchNormalization
from keras.layers import GlobalAveragePooling2D, Flatten, MaxPooling2D, Activation, ZeroPadding2D
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras import metrics
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from keras import backend as K
import tensorflow as tf

In [ ]:
model_base = InceptionV3(include_top = False, weights='imagenet', input_shape=(input_size, input_size, 3))

In [ ]:
model_base.summary()

In [ ]:
from imgaug import augmenters as iaa
def augment(image):
    augment_img = iaa.Sequential([
        iaa.OneOf([
            iaa.Fliplr(0.5), iaa.Crop(percent=(0, 0.1)),
            iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                       rotate=(-180, 180), shear=(-2, 2))])], random_order=True)

    image_aug = augment_img.augment_image(image)
    return image_aug

In [ ]:
def ProteinDataGenerator(generator, label, batch_size, augment_tf=None):

    index = 0
    while True:
        batch = []
        for i in range(batch_size):
            image = []
            temp = generator.next()[0].reshape(4, input_size, input_size)

            image.append(temp[2])
            image.append(temp[1])
            image.append(temp[0])

            image = np.dstack(image)
            
            if(augment_tf):
                image = augment(image)
                        
            batch.append(image)
            
        batch = np.asarray(batch, dtype=np.float16)
        
        if len(label) - index > batch_size:
            y = label[index : index + batch_size]
            y = y.iloc[:, 1:]
            index += batch_size
        elif len(label) - index < batch_size:
            y = pd.concat([label[index : len(label)], label[0 : batch_size - (len(label) - index)]])
            y = y.iloc[:, 1:]
            index = batch_size - (len(label) - index)
        else:
            y = label[index : index + batch_size]
            y = y.iloc[:, 1:]
            index = 0
        
        yield batch, y

In [ ]:
def f1(y_true, y_pred):
    
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [ ]:
train_label = pd.DataFrame([])
for k in tqdm(train_labels):
    train_label = train_label.append(labels_csv[labels_csv['Id'] == k])

In [ ]:
valid_label = pd.DataFrame([])
for k in tqdm(valid_labels):
    valid_label = valid_label.append(labels_csv[labels_csv['Id'] == k])

In [ ]:
train_label

In [ ]:
valid_label

In [ ]:
drop = 0.3
batch_size = 16

In [ ]:
for layer in model_base.layers:
    layer.trainable = True

In [ ]:
init = Input((input_size, input_size, 3))

x = BatchNormalization()(init)

x = model_base(x)

x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3, 3))(x)

x = Flatten()(x)

x = Dropout(drop)(x)

x = BatchNormalization()(x)
x = Dense(1024)(x)
x = LeakyReLU(0.1)(x)

x = Dropout(drop)(x)

x = BatchNormalization()(x)
x = Dense(512)(x)
x = LeakyReLU(0.1)(x)

x = Dropout(drop)(x)

x = Dense(28)(x)
x = Activation('sigmoid')(x)

model = Model(init, x)

In [ ]:
os.chdir('..')
model.load_weights("InceptionV3_352x352_RGB_400.h5")
print("Loaded model from disk")
os.chdir('train')

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=[f1, 'acc', 'categorical_accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit_generator(ProteinDataGenerator(train_generator, train_label, batch_size, augment_tf=True),
                    validation_data=ProteinDataGenerator(valid_generator, valid_label, batch_size, augment_tf=False),
                    validation_steps=60, steps_per_epoch=85, epochs=100)

In [ ]:
os.chdir('..')

In [ ]:
model.save_weights("InceptionV3_352x352_RGB_500.h5")
print("Saved model to disk")

In [ ]:
os.getcwd()

In [ ]:
#shutil.rmtree('test')

In [ ]:
#with zipfile.ZipFile('test.zip', 'r') as zip_ref:
#    zip_ref.extractall('test')

In [ ]:
submit = pd.read_csv('sample_submission.csv')

In [ ]:
#os.chdir('test')
#colors = ['red', 'green', 'blue', 'yellow']
#for i in submit['Id']:
#    os.mkdir(i)
#    for j in colors:
#        img = str(i) + '_' + str(j) +'.png'
#        path = i
#        shutil.move(img, path)
#os.chdir('..')

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory="test/",
    target_size=(input_size, input_size),
    color_mode="grayscale",
    batch_size=img_group,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

In [ ]:
prediction = []

In [ ]:
def TestProteinGenerator(generator, batch_size):
    batch = []
    for i in range(batch_size):
        image = []
        temp = generator.next()[0].reshape(4, input_size, input_size)

        image.append(2*temp[2]/3 + temp[3]/3)
        image.append(temp[1])
        image.append(2*temp[0]/3 + temp[3]/3)

        image = np.dstack(image)

        batch.append(image)

    batch = np.asarray(batch)

    return batch

In [ ]:
for i in tqdm(submit['Id']):
    x_test_temp = TestProteinGenerator(test_generator, 1)
    prediction.append(model.predict(x_test_temp))

In [ ]:
pred = np.vstack(prediction)

In [ ]:
prediction = []

In [ ]:
for i in range(0, 11702):
    label_predict = np.arange(28)[pred[i]>=0.2]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    prediction.append(str_predict_label)

In [ ]:
submit['Predicted'] = prediction

In [ ]:
submit.to_csv('submit_InceptionV3.csv', index=False)